# SWOT Feature Translation Service (FTS) Examples

This is the associated Jupyter Notebook containing four examples related to querying the SWOT Feature Translation Service and using its results to query NASA's Common Metadata Repository (CMR).

In [1]:
%%capture

import sys
!{sys.executable} -m pip install bs4 requests

In [2]:
from bs4 import BeautifulSoup
import requests
import json

## Exact SWOT Feature ID Matches in the SWOT Feature Translation Service

Below you can input some given SWOT Feature ID and query the Feature Translation Service based on it.

In [3]:
###################

# Mimicing a user querying exact matches with SWOT Feature ID "7541140001000000"

SWOT_FEATURE_ID = "7541140001000000"
EXACT = True

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/swot/{}?exact={}".format(SWOT_FEATURE_ID, EXACT))

# Load response from FTS
response = json.loads(r.text)

# Print all elements in HUC database that exactly match SWOT Feature ID "75411400010000"
print(json.dumps(response, indent = 4))

{
    "status": "200 OK",
    "hits": 1,
    "time": "2.093 ms.",
    "search on": {
        "parameter": "SWOT Feature ID",
        "exact": true
    },
    "results": {
        "7541140001000000": "point=-83.62575151546879,9.252305471126416"
    }
}


## Partial SWOT Feature ID Matches in the SWOT Feature Translation Service

In a similar fashion to the HUC FTS portion, you can query the service for partial matches to the SWOT Feature ID. This will return a string formatted to directly be used in a CMR query.

In [4]:
###################

# Mimicing a user querying partial matches with SWOT Feature ID "75411400010000"

SWOT_FEATURE_ID = "75411400010000"
EXACT = False

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/swot/{}?exact={}".format(SWOT_FEATURE_ID, EXACT))

# Load response from FTS
response = json.loads(r.text)

# Print all elements in HUC database that exactly match HUC "180500030105"
print(json.dumps(response, indent = 4))

{
    "status": "200 OK",
    "hits": 10,
    "time": "4.283 ms.",
    "search on": {
        "parameter": "SWOT Feature ID",
        "exact": false
    },
    "results": {
        "75411400010000": "point=-83.62575151546879,9.252305471126416&point=-83.62575352733344,9.252576533363468&point=-83.62564640094489,9.252848395107943&point=-83.62553927444125,9.253120256862395&point=-83.62543214782254,9.253392118626815&point=-83.62527045178793,9.253664380130763&point=-83.62510875554258,9.253936641627057&point=-83.62489208736692,9.254155090331341&point=-83.62467461434254,9.254265113998752&point=-83.62440257170697,9.25437553719889"
    }
}


## Query CMR by String

Below is an example of how the output of the SWOT Feature Translation Service portion can directly be used to query CMR. I'm using the given SWOT Feature ID to search for granules available through the Sentinel-1 mission. Currently this is through a string, however this isn't technically correct as we're _AND'ing_ spatial features together rather than _OR'ing_ them. 

As of now, CMR does not support POSTing a JSON body of spatial parameters, however this is an area actively being looked into.

In [5]:
###################

# Mimicing a user querying exact matches with SWOT Feature ID "75411400010000"

COLLECTION_ID = "C1522341104-NSIDC_ECS" # SMAP/Sentinel-1 L2 Radiometer/Radar 30-Second Scene 3 km EASE-Grid Soil Moisture V002
SWOT_FEATURE_ID = "75411400010000"
EXACT = False

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/swot/{}?exact={}".format(SWOT_FEATURE_ID, EXACT))

# Load response from FTS
response = json.loads(r.text)

geo_list = response['results'][SWOT_FEATURE_ID]
#print(geo_list)

# Query CMR
# --------- #

#cmr_response = requests.get("https://cmr.earthdata.nasa.gov/search/granules.json?{}&echo_collection_id=C1522341104-NSIDC_ECS&pretty=True".format(polygon))
cmr_response = requests.get("https://cmr.earthdata.nasa.gov/search/granules?{}&echo_collection_id={}&pretty=True".format(geo_list, COLLECTION_ID))

# --------- #

# Make it look nice
soup = BeautifulSoup(cmr_response.text, features = 'lxml')
print(soup.prettify())

<?xml version="1.0" encoding="UTF-8"?>
<html>
 <body>
  <results>
   <hits>
    287
   </hits>
   <took>
    164
   </took>
   <references>
    <reference>
     <name>
      SC:SPL2SMAP_S.002:142143638
     </name>
     <id>
      G1568764227-NSIDC_ECS
     </id>
     <location>
      https://cmr.earthdata.nasa.gov:443/search/concepts/G1568764227-NSIDC_ECS/6
     </location>
     <revision-id>
      6
     </revision-id>
    </reference>
    <reference>
     <name>
      SC:SPL2SMAP_S.002:135678909
     </name>
     <id>
      G1542789106-NSIDC_ECS
     </id>
     <location>
      https://cmr.earthdata.nasa.gov:443/search/concepts/G1542789106-NSIDC_ECS/8
     </location>
     <revision-id>
      8
     </revision-id>
    </reference>
    <reference>
     <name>
      SC:SPL2SMAP_S.002:135678915
     </name>
     <id>
      G1542789112-NSIDC_ECS
     </id>
     <location>
      https://cmr.earthdata.nasa.gov:443/search/concepts/G1542789112-NSIDC_ECS/8
     </location>
     <revision-id>

## Query CMR via JSON

As mentioned in the previous example, this is not currently possible, however I've added direct support for when it is. Once available, a proverbial lever can be flicked to allow querying via JSON. The code below is something that can be used once possible.

In [ ]:
###################

# Mimicing a user querying exact matches with SWOT Feature ID "75411400010000"

COLLECTION_ID = "C1522341104-NSIDC_ECS" # SMAP/Sentinel-1 L2 Radiometer/Radar 30-Second Scene 3 km EASE-Grid Soil Moisture V002
SWOT_FEATURE_ID = "75411400010000"
EXACT = False

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/swot/{}?exact={}".format(SWOT_FEATURE_ID, EXACT))

# Load response from FTS
response = json.loads(r.text)
json_response = response['results'][SWOT_FEATURE_ID]
#print(json_response)

# Query CMR
# --------- #

#cmr_response = requests.post("https://cmr.earthdata.nasa.gov/search/granules?echo_collection_id={}&pretty=True".format(COLLECTION_ID), data = json_response)
cmr_response = requests.post("https://cmr.earthdata.nasa.gov/search/granules?echo_collection_id={}&pretty=True".format(COLLECTION_ID), data = json_response)

# --------- #

# Make it look nice
soup = BeautifulSoup(cmr_response.text, features = 'lxml')
print(soup.prettify())